# **NMR chemical shift data as rich nuclear environment features**
Towards continuous atom typing

This allows us to do something quite powerful: predict any of the heads given the other information. Specifically of interest to enzyme design,
- Given an arbitrary point cloud with bond connections, pH, and Temperature, give the **chemical shift (deshielding) of each individual nucleus**


## Imports

In [1]:
import os
import pickle
import numpy as np
import pandas as pd
import pynmrstar
import MDAnalysis as mda

import matplotlib.pyplot as plt

from tqdm import tqdm

from os.path import exists

/home/northja/miniconda3/envs/ML/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from src.fileio import search_files, collect_dataset, load_dataset

## Example

In [ ]:
entry = pynmrstar.Entry.from_file(
    "/home/northja/projects/chem_phys/deepNMR/database/bmrb_entries_protein/bmr4424/bmr4424_3.str",
    convert_data_types=True
)

In [ ]:
cs_result_sets = []
tags = ['Comp_index_ID', 'Comp_ID', 'Atom_ID', 'Atom_type', 'Val', 'Val_err']
for chemical_shift_loop in entry.get_loops_by_category("Atom_chem_shift"):
    cs_result_sets.append(chemical_shift_loop.get_tag(tags))
#cs_result_sets

In [ ]:
chem_shifts = np.array(cs_result_sets[0])

In [ ]:
df = pd.DataFrame(
    data=chem_shifts, 
    columns=["res_idx", "res_id", "atom_type", "element", "chem_shift", "cs_error"]
)
df

In [ ]:
df["chem_shift"].hist()

### Merge with PDB structure

In [ ]:
tags = ["Accession_code"]

pdb_id = [db_loop.get_tag(tags) for db_loop in entry.get_loops_by_category("Assembly_db_link")]


In [ ]:
pdb_id

## Partial dataset

### Load dataset

In [3]:
files = search_files(
    directory="/home/northja/datasets/bmrb/bmrb_entries_protein/",
    extension=".str"
)
#files
len(files)

14449

In [4]:
collect_dataset(files=files)

  0%|          | 27/14449 [00:04<22:40, 10.60it/s] /home/northja/projects/chem_phys/deepNMR/src/fileio.py:93: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  chem_shifts = np.array(cs_result_sets)[0]
Couldn't convert tag data type because it is not in the dictionary: _Experiment.Details
  0%|          | 70/14449 [00:12<27:11,  8.82it/s]/home/northja/projects/chem_phys/deepNMR/src/fileio.py:93: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  chem_shifts = np.array(cs_result_sets)[0]
  1%|          | 91/14449 [00:14<19:41, 12.15it/s]/home/northja/projects/

In [5]:
(chemical_shifts, pdb_ids, element_sets, conditions) = load_dataset()

SUCCESS:	 Datafiles loaded from disk


In [9]:
import matplotlib.pyplot as plt

In [12]:
chemical_shifts[0]

,res_idx,res_id,atom_type,element,chem_shift,cs_error
0,7,GLY,HA2,H,3.934,0.030
1,7,GLY,HA3,H,3.934,0.030
2,7,GLY,C,C,173.706,0.300
3,7,GLY,CA,C,44.973,0.300
4,8,ASP,H,H,8.172,0.030
...,...,...,...,...,...,...
878,78,GLY,H,H,7.987,0.030
879,78,GLY,HA2,H,3.764,0.030
880,78,GLY,HA3,H,3.764,0.030
881,78,GLY,CA,C,46.249,0.300


In [ ]:
pdb_ids

In [13]:
element_sets

[{'C', 'H', 'N'},
 {'H'},
 {'C', 'H', 'N'},
 {'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'H'},
 {'C', 'H', 'N'},
 {'H'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N', 'P'},
 {'C', 'H', 'N'},
 {'H'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'H'},
 {'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H'},
 {'C', 'H', 'N'},
 {'H'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'H', 'N'},
 {'H'},
 {'C', 'H', 'N'},
 {'H'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'H', 'N'},
 {'H'},
 {'H'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'C', 'H', 'N'},
 {'H', 'N'},
 {'H'},
 {'H'},
 {'

In [17]:
items = [list(e) for e in element_sets]

In [20]:
set(sum(items, []))

{'C', 'F', 'H', 'N', 'P'}

### Analyze

In [ ]:
len(pdb_ids)

In [ ]:
counts = []

for i in pdb_ids:
    counts.append(len(i))

In [ ]:
### remove the non-PDB codes
only_pdbs = []

for i in pdb_ids:
    list_j = []
    for j in i:
        for k in j:
            if k is not None and len(k) == 4:
                list_j.append(k)
    only_pdbs.append(list_j)

In [ ]:
l = [len(p) for p in only_pdbs]

In [ ]:
# Plot of pdbs structures per NMR structure
plt.hist(l, bins=98)
plt.yscale('log')
plt.xscale('log')
plt.xlabel('Number of PDB structures')
plt.ylabel('Number of NMR shift sets')
plt.axvline(x=1, c='red')

In [ ]:
tensors = [t.squeeze() for t in chemical_shifts]

In [ ]:
num_nuclei = 0
for i, model in enumerate(tensors):
    num_nuclei = num_nuclei + len(model)
print(num_nuclei)

### Data finagling

In [ ]:
counter = 0

structured_pdbs = []

for p in pdb_ids:
    if len(p) is not 0:
        counter = counter + 1
        structured_pdbs.append(p[0])

print(counter)

In [ ]:
structured_pdbs

In [ ]:
counts = [len(i) for i in structured_pdbs]

In [ ]:
plt.hist(counts)

In [ ]:
#[i for i in [j for j in structured_pdbs]]

flat_list = [item for sublist in structured_pdbs for item in sublist]

In [ ]:
flat_list

In [ ]:
just_pdbs = []

for i in flat_list:
    if isinstance(i, str) and len(i) == 4:
        just_pdbs.append(i)

In [ ]:
len(just_pdbs)

### Structural analysis

#### Download data

In [ ]:
pdb_simple = [p[0] for p in pdb_ids]

In [ ]:
from src.fileio import download_dataset
datadir = "/home/northja/datasets/nmr_corresponding_structures/"

download_dataset(datadir=datadir, pdb_ids=pdb_simple)

#### Merge coordinates into chemical shift dataframes

In [ ]:
chemical_shifts[0]

In [ ]:
pdb_ids[0][0]

In [ ]:
file = "/home/northja/datasets/nmr_corresponding_structures/2KQK.pdb"

In [ ]:
import MDAnalysis as mda
from MDAnalysis.analysis import rms, align

In [ ]:
u = mda.Universe(file)

In [ ]:
u.select_atoms('protein')

In [ ]:
u.trajectory

### Merge with single structure

In [ ]:
best_structure = u.trajectory[0] # best-matched structure to NMR model

In [ ]:
atype = u.atoms.types

In [ ]:
resid = u.atoms.resids

In [ ]:
resnm = u.atoms.resnames

In [ ]:
aname = u.atoms.names

In [ ]:
for a in best_structure:
    print(a)
    #print(a)

In [ ]:
structure = np.array(best_structure).transpose()

In [ ]:
xs = structure[0]
ys = structure[1]
zs = structure[2]

In [ ]:
len(best_structure)

In [ ]:
atompos = np.stack(
    (resid.T, resnm.T, atype.T, aname.T, xs.T, ys.T, zs.T),
    axis=1
)

In [ ]:
atompos

In [ ]:
df = pd.DataFrame(
    data=atompos, 
    columns=["res_idx", "res_id", "element", "atom_type", "x", "y", "z"]
)

In [ ]:
df

In [ ]:
left = chemical_shifts[0]
right = df
with_shifts = pd.merge(left, right, how="left", on=["res_idx", "res_id", "element", "atom_type"])
all_atoms = pd.merge(left, right, how="right", on=["res_idx", "res_id", "element", "atom_type"])

In [ ]:
all_atoms

In [ ]:
from src.fileio import pickle_variable

In [ ]:
pickle_variable(variable=all_atoms, varname="all_atoms.pkl")

### Bond graph

In [119]:
from MDAnalysis.topology.guessers import guess_bonds

In [122]:
bondlist = guess_bonds(atoms=u.atoms, coords=u.coord)

In [127]:
bonds = np.array(bondlist).T

In [129]:
pickle_variable(variable=bonds, varname="bondlist.pkl")

### RMSF analysis

In [ ]:
average = align.AverageStructure(u, u, select='protein and name CA',
                                 ref_frame=0).run()
ref = average.universe

In [ ]:
aligner = align.AlignTraj(u, ref,
                          select='protein and name CA',
                          in_memory=True).run()

In [ ]:
c_alphas = u.select_atoms('protein and name CA')
all_Atoms = u.select_atoms('protein')
R = rms.RMSF(all_Atoms).run() 

In [ ]:
rmsf = R.rmsf

In [ ]:
rmsf

In [ ]:
len(rmsf)

In [ ]:
stability = 1/rmsf

In [ ]:
stability

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.hist(stability, bins=20)
plt.xlabel("Stability $s=1/rmsf$")
plt.ylabel("Counts")

In [ ]:
protein = u.select_atoms("protein")
protein.write("protein.pdb")